In [1]:
require 'btp'


Install the spreadsheet gem version ~>1.1.1 for using spreadsheet functions.

Install the mechanize gem version ~>2.7.5 for using mechanize functions.


true

In [2]:
unigram_df = BTP::Unigram.new('../../spec/fixtures/finnish/finnish-task1-train').parse

,character,left_position,right_position,count
0,a,0,-14,32
1,a,1,-13,72
2,l,2,-12,31
3,l,3,-11,25
4,o,4,-10,39
5,i,5,-9,46
6,t,6,-8,25
7,t,7,-7,24
8,a,8,-6,40
9,i,9,-5,102


In [3]:
unigram_df.sort([:count], ascending: false)

,character,left_position,right_position,count
35,i,9,-1,661
88,i,8,-1,548
487,i,10,-1,498
21,i,7,-1,412
30,t,8,-2,332
79,i,11,-1,309
155,t,9,-2,297
452,i,6,-4,275
143,i,12,-1,267
200,i,4,-1,255


In [13]:
chars = unigram_df[:character].uniq.to_a
merged_distribution = Daru::Vector.new(chars.map { |char| [char, unigram_df.where(unigram_df[:character].eq(char))[:count].sum] }.sort_by { |k, _v| k }.to_h)

-,86
a,7047
b,541
c,70
d,959
e,4021
f,497
g,503
h,1352
i,11551
j,564
